In [1]:
import pandas as pd
import numpy as np
from pyxlsb import convert_date

In [2]:
footfall = pd.read_excel('data/footfall.xlsx')
occupancy_costs = pd.read_excel('data/OccupancyCosts.xlsx')
tenancy_schedule = pd.read_excel('data/The Glades Tenancy Schedule.xlsx',
                                skiprows = 1)

In [3]:
name_dict = {'black cab coffee co': 'black cab coffee', 
             'apple store': 'apple',
            'and cut hair & beauty': 'and cut', 
            'caffè nero': 'caffe nero', 
            'crew clothing co.': 'crew clothing co', 
            'crystal palace football club': 'crystal palace f.c',
             'intime ': 'intime',
             'suit direct ': 'suit direct'
            }

In [4]:
occupancy_costs['mainCategory'] = occupancy_costs['Category']\
                                    .apply(lambda x: x.split(">>")[0])
occupancy_costs['Name'] = occupancy_costs['Name']\
                .apply(lambda x: x.lower() if isinstance(x, str) else '')
occupancy_costs = occupancy_costs.replace(name_dict)

In [5]:
tenancy_schedule.rename(columns = {'Shop': 'Name'}, inplace = True)
tenancy_schedule['Name'] = tenancy_schedule['Name']\
                .apply(lambda x: x.lower() if isinstance(x, str) else '')
tenancy_schedule = tenancy_schedule.replace(name_dict)

In [6]:
lease_expiries = pd.read_excel('data/lease_expiries.xlsx',
                               usecols=lambda x: 'Unnamed' not in x,
                               skiprows = 1, 
                               engine = 'pyxlsb')

lease_expiries.rename(columns = {'Name':'Number', 'Shop':'Name'},
                      inplace = True)

lease_expiries['Name'] = lease_expiries['Name']\
                            .apply(lambda x: x.lower() if isinstance(x, str) else '')

lease_expiries = lease_expiries.replace(name_dict)

In [7]:
fact_table = occupancy_costs.merge(tenancy_schedule,
                                   on = 'Name', how = 'outer')\
                            .merge(lease_expiries, 
                                   on = 'Name',
                                   how = 'outer')\
                            .sort_values(by = 'Name') \
                            .drop_duplicates()


In [8]:
clean_table = fact_table[fact_table.Name !=''].sort_values(by='Name')
clean_table['rent_review'] = clean_table['Rent Type Of Review'].apply(lambda x: 1 if isinstance(x, str) else 0)

In [9]:
fact_table.groupby(['Floor', 'Let Status']).count()['Name']

Floor  Let Status       
Lower  Let                   75
       Let [Shop Closed]      1
       Vacant               312
Upper  Let                   46
       Let [Shop Closed]      1
       Vacant               288
Name: Name, dtype: int64

In [10]:
clean_table.groupby(['Floor', 'rent_review']).count()['Name']

Floor  rent_review
Lower  0              37
       1              39
Upper  0              25
       1              22
Name: Name, dtype: int64

In [11]:
#what is the percentage of selling space?
# sell that somebody can sell in 

In [12]:
lease_expiries.groupby(['Floor', 'Let Status']).count()['Name']

Floor  Let Status       
Lower  Let                  72
       Let [Shop Closed]     1
       Vacant               13
Upper  Let                  43
       Let [Shop Closed]     1
       Vacant               12
Name: Name, dtype: int64

In [13]:
#find interesting aggregations to make of the datasets
#maybe make the visualizations myself?

#look at data for visitors and community, decide whether 
#it is feasible to get them in the next three/four weeks

In [14]:
clean_table.groupby('mainCategory').count()['Name']

mainCategory
Clothing & Accessories                 8
Clothing & Accessories                37
Department & Variety Stores            4
Health & Beauty                        9
Health & Beauty                       18
Household & Electrical & Hardware      2
Household & Electrical & Hardware      3
Leisure                                1
Multimedia & Gifts                     4
Multimedia & Gifts                    13
Other Retail                           1
Other Retail                           3
Restaurants & Bars                     6
Restaurants & Bars                    11
Services                               2
Services                               1
Name: Name, dtype: int64

In [15]:
clean_table.head()

,Name,DateOpened,Category,GrossSqFt,Rent_x,ServiceCharge,Insurance,Rates,Total,TotalPerGrossSqFt,...,Floor,Number,Unit Type,Let Status,Lease Review,Break Clause,Lease Expiry_y,Service Charge,Rent_y,rent_review
0,ada & albert,2020-12-16,Clothing & Accessories >> Women's Wear >> Ladi...,NaN,NaN,NaN,NaN,NaN,£0.00,NaN,...,Lower,58,NaN,Let,NaN,NaN,NaN,NaN,NaN,0
1,aldo,NaT,Clothing & Accessories >> Shoes >> Shoes,2229.0,"£45,000.00","£26,400.93","£1,028.80","£59,160.00","£131,589.73",£59.04,...,Lower,57,NaN,Let,NaN,Rolling break from 17/06/2016,43998.0,26400.93,45000.0,0
2,and cut,NaT,Health & Beauty >> Hairdressing,475.0,"£20,000.00","£5,023.83",£271.18,"£7,680.00","£32,975.01",£69.42,...,Lower,77,NaN,Let,NaN,NaN,44690.0,5023.83,20000.0,0
3,apple,NaT,Household & Electrical & Hardware,4935.0,"£315,000.00","£57,163.00","£3,053.00","£101,065.00","£476,281.00",£96.51,...,Lower,087/088/094,NaN,Let,43007.0,NaN,44832.0,52396.65,315000.0,1
4,auntie anne's,NaT,Restaurants & Bars >> Fast Food Restaurant,406.5,"£56,000.00","£4,378.79",£308.61,"£20,582.75","£81,270.15",£199.93,...,Lower,25,NaN,Let,42276.0,NaN,44102.0,4378.79,56000.0,1


In [16]:
dates = ['DateOpened', 'Lease Expiry_x', 'Lease Expiry_y', 'Lease Review']

In [19]:
clean_table[dates] = clean_table[dates].fillna(0)

In [20]:
clean_table[dates].head()

,DateOpened,Lease Expiry_x,Lease Expiry_y,Lease Review
0,2020-12-16 00:00:00,0,0.0,0.0
1,0,16/06/2020,43998.0,0.0
2,0,09/05/2022,44690.0,0.0
3,0,28/09/2022,44832.0,43007.0
4,0,28/09/2020,44102.0,42276.0


In [25]:
clean_table['Lease Expiry_y'] = clean_table['Lease Expiry_y'].apply(lambda x: format(convert_date(x), '%Y-%m-%d'))


TypeError: unsupported format string passed to NoneType.__format__

In [26]:
clean_table['Lease Review'] = clean_table['Lease Review'].apply(lambda x: format(convert_date(x), '%Y-%m-%d'))

In [27]:
clean_table[dates].head()

,DateOpened,Lease Expiry_x,Lease Expiry_y,Lease Review
0,2020-12-16 00:00:00,0,1900-01-01,1900-01-01
1,0,16/06/2020,2020-06-16,1900-01-01
2,0,09/05/2022,2022-05-09,1900-01-01
3,0,28/09/2022,2022-09-28,2017-09-29
4,0,28/09/2020,2020-09-28,2015-09-29


In [28]:
clean_table = clean_table.drop(['Lease Expiry_x'], axis=1)

In [29]:
clean_table.head()

,Name,DateOpened,Category,GrossSqFt,Rent_x,ServiceCharge,Insurance,Rates,Total,TotalPerGrossSqFt,...,Floor,Number,Unit Type,Let Status,Lease Review,Break Clause,Lease Expiry_y,Service Charge,Rent_y,rent_review
0,ada & albert,2020-12-16 00:00:00,Clothing & Accessories >> Women's Wear >> Ladi...,NaN,NaN,NaN,NaN,NaN,£0.00,NaN,...,Lower,58,NaN,Let,1900-01-01,NaN,1900-01-01,NaN,NaN,0
1,aldo,0,Clothing & Accessories >> Shoes >> Shoes,2229.0,"£45,000.00","£26,400.93","£1,028.80","£59,160.00","£131,589.73",£59.04,...,Lower,57,NaN,Let,1900-01-01,Rolling break from 17/06/2016,2020-06-16,26400.93,45000.0,0
2,and cut,0,Health & Beauty >> Hairdressing,475.0,"£20,000.00","£5,023.83",£271.18,"£7,680.00","£32,975.01",£69.42,...,Lower,77,NaN,Let,1900-01-01,NaN,2022-05-09,5023.83,20000.0,0
3,apple,0,Household & Electrical & Hardware,4935.0,"£315,000.00","£57,163.00","£3,053.00","£101,065.00","£476,281.00",£96.51,...,Lower,087/088/094,NaN,Let,2017-09-29,NaN,2022-09-28,52396.65,315000.0,1
4,auntie anne's,0,Restaurants & Bars >> Fast Food Restaurant,406.5,"£56,000.00","£4,378.79",£308.61,"£20,582.75","£81,270.15",£199.93,...,Lower,25,NaN,Let,2015-09-29,NaN,2020-09-28,4378.79,56000.0,1


In [30]:
clean_table.to_csv('~/Desktop/clean_table_clean.csv')